# Pruebas

+ En las definiciones de las funciones en LennardGas ya no se suponen valores predeterminados para los parámetros, siempre hay que darlos explicitamente para realizar pruebas.

In [8]:
push!(LOAD_PATH, pwd())
using LennardGas: flotante_a_entero, entero_a_flotante, fluctuacion_gaussiana, vector_fuerzas, paso_verlet, evolucion

In [3]:
#Parámetros
r_c = 2.5
L   = r_c * 10
cajitas = 2^60
h = 0.005

0.005

In [5]:
#Condición inicial (en Float64)
inicial = randn(1200) .+ 10.0
segundo = fluctuacion_gaussiana(inicial, 0.1);

In [7]:
#Condición inicial (en Int64)
X0 = flotante_a_entero(inicial, L, cajitas)
X1 = flotante_a_entero(segundo, L, cajitas);

In [12]:
@time vector_fuerzas(X0 , L, cajitas, r_c, h);

  0.044106 seconds (2.14 k allocations: 205.672 KB)


In [14]:
@time X2 = paso_verlet(X0, X1, L, cajitas, r_c, h);

  0.046773 seconds (2.15 k allocations: 219.188 KB)


In [16]:
X_0 = paso_verlet(X2, X1, L, cajitas, r_c, h);
X0 == X_0

false

#### MUY poco reversible por ahora...

In [19]:
pasos = 1000
@time evolucion(X0, X1, pasos, L, cajitas, r_c, h)

1002x1200 Array{Int64,2}:
  419992543260070656   528878706644030848  …   530226508924207744
  422924369281512192   542580981707065280      529599434896077760
  105438218899989371   467835267733275386       13706303943223823
  633796689536837406   325326467081005895      557011931080453818
  219976877440153754  1118633657697007608     1127730271346327092
 1073750539997885619   256750735450535277  …   506629436749696762
   61634887867920505   479213536332457179      988067291568013476
   44098891702928748    62256291340502321      176921033171623035
  149832382874889671   603723322841692171      188987909547420756
  343763610305489384   116781338534432909      554896371745165695
  837359679645399058   837285777126417126  …   145859276690930405
  865561665595357417   638431090448714746      846615947351764516
  262640251694924056   961226161037286911      686168552876981431
                   ⋮                       ⋱                     
  701912436571804278   651829726413138027  …   116

0.796710 seconds (3.44 M allocations: 264.471 MB, 2.79% gc time)
